In [166]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as plticker
import numpy as np
import sklearn.model_selection as ms

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

plt.style.use('seaborn-talk')
plt.style.use('ggplot')


In [167]:
fbsplays14 = pd.read_csv('fbsplays/fbsplays_14.csv')
fbsplays15 = pd.read_csv('fbsplays/fbsplays_15.csv')
fbsplays16 = pd.read_csv('fbsplays/fbsplays_16.csv')
fbsplays17 = pd.read_csv('fbsplays/fbsplays_17.csv')
fbsplays18 = pd.read_csv('fbsplays/fbsplays_18.csv')
fbsplays19 = pd.read_csv('fbsplays/fbsplays_19.csv')
fbsplays20 = pd.read_csv('fbsplays/fbsplays_20.csv')
fbsplays21 = pd.read_csv('fbsplays/fbsplays_21.csv')
fbsplays22 = pd.read_csv('fbsplays/fbsplays_22.csv')

In [170]:
def offense_calcs(x):
    names = {'game_id': str(x['game_id'].unique()),
             'home': str(x['home'].unique()),
             'away': str(x['away'].unique()),
             'Off. Conf': str(x['offense_conference'].unique()),
             'Def. Conf': str(x['defense_conference'].unique()),
             'Plays': x[x['rush']==1]['rush'].count() + x[x['pass']==1]['pass'].count(),
             'Rush Atts': x[x['rush'] == 1]['yards_gained'].count(),
             'Rush Yards': x[x['rush'] == 1]['yards_gained'].sum(),
             'Pass Atts': x[x['pass'] == 1]['yards_gained'].count(),
             'Pass Yards': x[(x['pass'] == 1) & (x['play_type']!= 'Sack')]['yards_gained'].sum(),
             'Off. Drives' : len(x['drive_id'].unique()), # Number of Offensive Drives vs Defense
             'Turnover': x[x['play_type'].str.contains('Interception')]['play_type'].count() + x[x['play_type'].str.contains('Opponent')]['play_type'].count(), # Number of Turnovers
             'Field Goal Made': x[x['play_type'].str.contains('Field Goal Good')]['play_type'].count(),
             'Field Goal Attempt': x[x['play_type'].str.contains('Field Goal')]['play_type'].count()
            }
    return pd.Series(names)



In [171]:
offense14 =  fbsplays14.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense15 =  fbsplays15.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense16 =  fbsplays16.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense17 =  fbsplays17.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense18 =  fbsplays18.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense19 =  fbsplays19.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense20 =  fbsplays20.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense21 =  fbsplays21.groupby(['offense','defense'], sort = False).apply(offense_calcs)
offense22 =  fbsplays22.groupby(['offense','defense'], sort = False).apply(offense_calcs)


In [172]:
score14 =  fbsplays14.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score15 =  fbsplays15.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score16 =  fbsplays16.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score17 =  fbsplays17.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score18 =  fbsplays18.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score19 =  fbsplays19.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score20 =  fbsplays20.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score21 =  fbsplays21.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)
score22 =  fbsplays22.groupby(['offense','defense'], sort = False)['offense_score'].nth(-1)

In [208]:
df14 = pd.merge(offense14, score14, on=['offense','defense'])
df15 = pd.merge(offense15, score15, on=['offense','defense'])
df16 = pd.merge(offense16, score16, on=['offense','defense'])
df17 = pd.merge(offense17, score17, on=['offense','defense'])
df18 = pd.merge(offense18, score18, on=['offense','defense'])
df19 = pd.merge(offense19, score19, on=['offense','defense'])
df20 = pd.merge(offense20, score20, on=['offense','defense'])
df21 = pd.merge(offense21, score21, on=['offense','defense'])
df22 = pd.merge(offense22, score22, on=['offense','defense'])


In [209]:
df = [df14, df15, df16, df17, df18, df19, df20, df21, df22]

for df in df:
    df['defense_score'] = df.swaplevel(0,1)['offense_score']
    df['defense_score'] = df['defense_score'].fillna(0)

### Points per Drive, Win, Score Difference (Offense - Defense)

In [210]:
df = [df14, df15, df16, df17, df18, df19, df20, df21, df22]

for df in df:
    df['PPD'] = df['offense_score']/df['Off. Drives']
    df['Win'] = df['offense_score'] > df['defense_score']
    df['Win'] = df['Win'].astype(int)
    df['score_diff'] = df['offense_score'] - df['defense_score']
    df['OT'] = df['score_diff'] == 0
    df['OT'] = df['OT'].astype(int)


In [211]:
df14 = df14.reset_index()
df15 = df15.reset_index()
df16 = df16.reset_index()
df17 = df17.reset_index()
df18 = df18.reset_index()
df19 = df19.reset_index()
df20 = df20.reset_index()
df21 = df21.reset_index()
df22 = df22.reset_index()


In [212]:
df_list = [df14,df15,df16,df17,df18,df19,df20,df21,df22]

dict = {'offense_score' : 'home_score','defense_score' : 'away_score'}

for df in df_list:
    df.rename(dict,axis = 'columns',inplace = True)


In [222]:
df14.drop(df14.index[114],inplace = True)
df14.drop(df14.index[1199],inplace = True)

In [233]:
df_list = [df14,df15,df16,df17,df18,df19,df20,df21,df22]

def change_values(df):
    df.loc[df['home'] == '['+"'"+df['offense']+"'"+']', 'home'] = 1
    df.loc[df['away'] == '['+"'"+df['defense']+"'"+']', 'away'] = -1
    return df

dict = {'home': 'LocOff', 'away': 'LocDef','home_score': 'offense_score', 'away_score': 'defense_score'}

for df in df_list:
    change_values(df)
    df.rename(dict,axis = 'columns',inplace = True)
    df['LocOff'] = pd.to_numeric(df['LocOff'], errors='coerce').fillna(-1).astype(int)
    df['LocDef'] = pd.to_numeric(df['LocDef'], errors='coerce').fillna(1).astype(int)
    
    
    


In [234]:
df14 = df14.reset_index(drop = True)
df15 = df15.reset_index(drop = True)
df16 = df16.reset_index(drop = True)
df17 = df17.reset_index(drop = True)
df18 = df18.reset_index(drop = True)
df19 = df19.reset_index(drop = True)
df20 = df20.reset_index(drop = True)
df21 = df21.reset_index(drop = True)
df22 = df22.reset_index(drop = True)


In [235]:
df14.to_csv('games14.csv')
df15.to_csv('games15.csv')
df16.to_csv('games16.csv')
df17.to_csv('games17.csv')
df18.to_csv('games18.csv')
df19.to_csv('games19.csv')
df20.to_csv('games20.csv')
df21.to_csv('games21.csv')
df22.to_csv('games22.csv')